## Training Yolov7

In [7]:
# run this cell to begin training
!python train.py --batch 8 --cfg cfg/training/yolov7.yaml \
--epochs 300 --data ./tomato/data.yaml --weights 'yolov7.pt' \
--device 0 --name yolov7-tomato --hyp data/hyp.scratch.p6.yaml --workers 2

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)


YOLOR  568ae86b torch 1.12.1+cu113 CUDA:0 (NVIDIA GeForce RTX 2060, 6143.5625MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7.yaml', data='./drinks-1/data.yaml', device='0', entity=None, epochs=300, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.p6.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov7-drinks', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\yolov7-drinks3', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights="'yolov7.pt'", workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup

## Resume

In [7]:
!python train.py --batch 8 --cfg cfg/training/yolov7.yaml \
--epochs 300 --data ./tomato/data.yaml --weights 'yolov7.pt' \
--device 0 --name yolov7-tomato --hyp data/hyp.scratch.p6.yaml \
--resume ./runs/train/exp/weights/last.pt

[Errno 2] No such file or directory: '/home/ubuntu/yolov7/yolov7'
/media/ubuntu/viscom/yolov7-training/yolov7
Resuming training from ./runs/train/exp/weights/last.pt
YOLOR 🚀 v0.1-70-g4c207e1 torch 1.10.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3060, 12051.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='./drinks-1/data.yaml', device='0', entity=None, epochs=300, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=True, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights='./runs/train/exp/weights/last.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', v

## Evaluation

In [6]:
!python detect.py --weights runs/train/yolov7-tomato3/weights/best.pt --conf 0.1 --source ./tomato/test/images

Namespace(weights=['runs/train/yolov7-tomato3/weights/best.pt'], source='./tomato/test/images', img_size=640, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

 The image with the result is saved in: runs\detect\exp\IMG_8470.jpg
 The image with the result is saved in: runs\detect\exp\IMG_8509.jpg
 The image with the result is saved in: runs\detect\exp\IMG_8510.jpg
 The image with the result is saved in: runs\detect\exp\IMG_8511.jpg
 The image with the result is saved in: runs\detect\exp\IMG_8551.jpg
 The image with the result is saved in: runs\detect\exp\IMG_8556.jpg
 The image with the result is saved in: runs\d

YOLOR  07528c5 torch 1.12.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3060, 12287.375MB)

c:\Users\z230\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36487166 parameters, 6194944 gradients, 103.2 GFLOPS


In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1
    

## Save weights

In [ ]:
# optional, zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

## OPTIONAL: Active Learning Example


Khi quá trình đào tạo đầu tiên của chúng tôi hoàn tất, chúng tôi nên sử dụng mô hình của mình để giúp xác định hình ảnh nào có vấn đề nhất để điều tra, chú thích và cải thiện tập dữ liệu của chúng tôi (và do đó, mô hình).

Để làm điều đó, chúng tôi có thể thực thi mã tự động tải hình ảnh trở lại tập dữ liệu được lưu trữ của chúng tôi nếu hình ảnh là một lớp cụ thể hoặc dưới ngưỡng tin cậy nhất định.

In [ ]:
# setup access to your workspace
rf = Roboflow(api_key="tIC0Djr3IIEJtHsuG1A4")                               # used above to load data
inference_project =  rf.workspace().project("drinks-jicb6")    # used above to load data
model = inference_project.version(1).model

upload_project = rf.workspace().project("drinks-jicb6")

print("inference reference point: ", inference_project)
print("upload destination: ", upload_project)

In [ ]:
# example upload: if prediction is below a given confidence threshold, upload it 

confidence_interval = [10,70]                                   # [lower_bound_percent, upper_bound_percent]

for prediction in predictions:                                  # predictions list to loop through
  if(prediction['confidence'] * 100 >= confidence_interval[0] and 
          prediction['confidence'] * 100 <= confidence_interval[1]):
        
          # upload on success!
          print(' >> image uploaded!')
          upload_project.upload(image, num_retry_uploads=3)     # upload image in question